# 第9章 単純ベイズ分類器

## 準備

In [1]:
import pprint
import numpy as np
from fractions import Fraction

# 上位K件
TOP_K = 3
# スムージングパラメタ
ALPHA = 1
# クラス数
N = 2
# 各特徴量がとりうる値のユニーク数
M = [2, 2, 2, 2, 2, 2]
# しきい値
THETA = 0.5

Du = np.array([
               [1, 0, 0, 0, 1, 0, +1],
               [0, 1, 0, 0, 1, 0, +1],
               [1, 1, 0, 0, 1, 0, +1],
               [1, 0, 0, 1, 1, 0, +1],
               [1, 0, 0, 0, 0, 1, +1],
               [0, 1, 0, 1, 0, 1, +1],
               [0, 0, 1, 0, 1, 0, -1],
               [0, 0, 1, 1, 1, 0, -1],
               [0, 1, 0, 0, 1, 1, -1],
               [0, 0, 1, 0, 0, 1, -1],
               [1, 1, 0, 1, 1, 0, np.nan],
               [0, 0, 1, 0, 1, 1, np.nan],
               [0, 1, 1, 1, 1, 0, np.nan],
])
I = np.arange(Du.shape[0])
x = Du[:,:-1]
ru = Du[:,-1]

Iu = I[~np.isnan(ru)]
Iu_not = np.setdiff1d(I, Iu)
DuL = Du[Iu]
xL = x[Iu]
ruL = ru[Iu]
DuU = Du[Iu_not]
xU = x[Iu_not]

## 問題設定

## 事前確率

### 01 評価値がrとなる事前確率（分子）
### 02 評価値がrとなる事前確率（分母）

In [2]:
def P_prior(r):
    """
    評価値がrとなる事前確率を返す。

    Parameters
    ----------
    r : int
        評価値

    Returns
    -------
    Fraction
        事前確率
    """
    # 01
    num = DuL[ruL==r].shape[0]
    # 02
    den = DuL.shape[0]
    prob = Fraction(num, den, _normalize=False)
    return prob

In [3]:
r = +1
print('P(R={:+}) = {}'.format(r, P_prior(r)))
r = -1
print('P(R={:+}) = {}'.format(r, P_prior(r)))

P(R=+1) = 6/10
P(R=-1) = 4/10


## 特徴量kに関する条件付き確率

### 03 特徴量kに関する条件付き確率（分子）
### 04 特徴量kに関する条件付き確率（分母）

In [4]:
def P_cond(i, k, r):
    """
    評価値がrとなる条件下でアイテムiの特徴量kに関する条件付き確率を返す。

    Parameters
    ----------
    i : int
        アイテムiのID
    k : int
        特徴量kのインデックス
    r : int
        評価値

    Returns
    -------
    Fraction
        条件付き確率
    """
    # 03
    num = DuL[ruL==r][xL[:,k][ruL==r]==x[i,k]].shape[0]
    # 04
    den = DuL[ruL==r].shape[0]
    prob = Fraction(num, den, _normalize=False)
    return prob

In [5]:
i = 10
k = 0
r = +1
print('P(X{}=x{},{}|R={:+}) = {}'.format(k, i, k, r, P_cond(i, k, r)))
r = -1
print('P(X{}=x{},{}|R={:+}) = {}'.format(k, i, k, r, P_cond(i, k, r)))

P(X0=x10,0|R=+1) = 4/6
P(X0=x10,0|R=-1) = 0/4


## 嗜好予測

### 05 好き嫌いの確率

In [6]:
def P(i, r):
    """
    アイテムiの評価値がrとなる確率を返す。

    Parameters
    ----------
    i : int
        アイテムiのID
    r : int
        評価値

    Returns
    -------
    Fraction
        事前確率
    list of Fraction
        各特徴量に関する条件付き確率
    float
        好き嫌いの確率
    """
    pp = P_prior(r)
    pk = [P_cond(i, k, r) for k in range(0, x.shape[1])]
    # 05
    prob = float(pp * np.prod(pk))
    return pp, pk, prob

In [7]:
i = 10
r = +1
pp, pk, prob = P(i, r)
left = 'P(R={:+}|'.format(r) + ','.join(map(str, map(int, x[i]))) + ')'
right = str(pp) + '×' + '×'.join(map(str, pk))
print('{} = {} = {:.3f}'.format(left, right, prob))

r = -1
pp, pk, prob = P(i, r)
left = 'P(R={:+}|'.format(r) + ','.join(map(str, map(int, x[i]))) + ')'
right = str(pp) + '×' + '×'.join(map(str, pk))
print('{} = {} = {:.3f}'.format(left, right, prob))

P(R=+1|1,1,0,1,1,0) = 6/10×4/6×3/6×6/6×2/6×4/6×4/6 = 0.030
P(R=-1|1,1,0,1,1,0) = 4/10×0/4×1/4×1/4×1/4×3/4×2/4 = 0.000


## ラプラススムージング

### 06 評価値がrとなる事前確率（分子）（ラプラススムージングあり）
### 07 評価値がrとなる事前確率（分母）（ラプラススムージングあり）
### 08 特徴量kに関する条件付き確率（分子）
### 09 特徴量kに関する条件付き確率（分母）

In [8]:
def P_prior(r):
    """
    評価値がrとなる事前確率を返す。

    Parameters
    ----------
    r : int
        評価値

    Returns
    -------
    Fraction
        事前確率
    """
    # 06
    num = DuL[ruL==r].shape[0] + ALPHA
    # 07
    den = DuL.shape[0] + ALPHA * N
    prob = Fraction(num, den, _normalize=False)
    return prob

In [9]:
r = +1
print('P(R={:+}) = {}'.format(r, P_prior(r)))
r = -1
print('P(R={:+}) = {}'.format(r, P_prior(r)))

P(R=+1) = 7/12
P(R=-1) = 5/12


In [10]:
def P_cond(i, k, r):
    """
    評価値がrとなる条件下でアイテムiの特徴量kに関する条件付き確率を返す。

    Parameters
    ----------
    i : int
        アイテムiのID
    k : int
        特徴量kのインデックス
    r : int
        評価値

    Returns
    -------
    Fraction
        条件付き確率
    """
    # 08
    num = DuL[ruL==r][xL[:,k][ruL==r]==x[i,k]].shape[0] + ALPHA
    # 09
    den = DuL[ruL==r].shape[0] + ALPHA * M[k]
    prob = Fraction(num, den, _normalize=False)
    return prob

In [11]:
i = 10
k = 0
r = +1
print('P(X{}=x{},{}|R={:+}) = {}'.format(k, i, k, r, P_cond(i, k, r)))
r = -1
print('P(X{}=x{},{}|R={:+}) = {}'.format(k, i, k, r, P_cond(i, k, r)))

P(X0=x10,0|R=+1) = 5/8
P(X0=x10,0|R=-1) = 1/6


## 推薦

In [12]:
def score(u, i):
    """
    スコア関数：ユーザuのアイテムiに対するスコアを返す。

    Parameters
    ----------
    u : int
        ユーザuのID（ダミー）
    i : int
        アイテムiのID

    Returns
    -------
    float
        スコア
    """
    # 10
    _, _, prob_p = P(i, +1)
    _, _, prob_n = P(i, -1)
    scr = prob_p / (prob_p + prob_n)
    return scr

In [13]:
def order(u, I):
    """
    順序付け関数：アイテム集合Iにおいて、ユーザu向けの推薦リストを返す。

    Parameters
    ----------
    u : int
        ユーザuのID
    I : ndarray
        アイテム集合

    Returns
    -------
    list
        タプル(アイテムID: スコア)を要素にした推薦リスト
    """
    scores = {i: score(u, i) for i in I}
    # 11
    scores = {i:scr for i,scr in scores.items() if scr >= THETA}
    rec_list = sorted(scores.items(), key=lambda x:x[1], reverse=True)[:TOP_K]
    return rec_list

### 10 ユーザuのアイテムiに対するスコア

In [14]:
u = 0
scores = {i: score(u, i) for i in Iu_not}
print('scores = ')
pprint.pprint(scores)

scores = 
{10: 0.9646054787625311, 11: 0.05517691284650013, 12: 0.18936236007174223}


### 11 推薦リスト

In [15]:
u = 0
rec_list = order(u, Iu_not)
print('rec_list = ')
for i, scr in rec_list:
    print('{}: {:.3f}'.format(i, scr))

rec_list = 
10: 0.965
